In [2]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

2024-07-23 09:12:51.801298: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-23 09:12:51.965671: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-23 09:12:51.965757: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-23 09:12:51.987655: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-23 09:12:52.034246: I tensorflow/core/platform/cpu_feature_guar

In [3]:
import csv

In [4]:
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

In [5]:
images_path = "/home/abhinav/Documents/Work/2 Hobby_projects/Datasets/Aiplanes_dataset_for_rcnn/Images"
label_path = "/home/abhinav/Documents/Work/2 Hobby_projects/Datasets/Aiplanes_dataset_for_rcnn/Airplanes_Annotations"

In [6]:
#there are 700 images with name starting with airplane, and 33 with numbers
img_bb = {}

In [8]:
for label in os.listdir(label_path):
    if label.split("_")[0] == "airplane":
        try:
            file = pd.read_csv(label_path+"/"+label)
            # print("label: ", label)
            # print("len of file: ", len(file))
            labels_for_image = []
            for row in file.iterrows():
                # for k in row:
                #     print(row[k][0].split(" "))
                bb_each = list(row[1])[0].split(" ")
                for j in range(len(bb_each)):
                    bb_each[j] = eval(bb_each[j])

                labels_for_image.append(bb_each)
            img_bb[label] = labels_for_image
        except Exception as e:
            print(e)
            print("Error in :", label)

In [12]:
print("Total images for training: ", len(img_bb))

Total images for training:  700


In [11]:
for i in img_bb:
    print("Image name: ", i)
    print("Number of labels: ", len(img_bb[i]))
    print("Labels: ", img_bb[i])

Image name:  airplane_201.csv
Number of labels:  26
Labels:  [[22, 78, 31, 90], [29, 69, 48, 81], [44, 65, 59, 76], [58, 57, 68, 70], [122, 16, 138, 26], [134, 9, 146, 21], [93, 35, 102, 46], [79, 41, 93, 56], [69, 48, 79, 59], [27, 110, 39, 121], [38, 102, 48, 111], [48, 97, 62, 105], [61, 89, 76, 97], [73, 82, 86, 91], [87, 75, 96, 84], [106, 69, 118, 79], [118, 53, 133, 65], [133, 47, 145, 58], [144, 39, 154, 48], [116, 238, 127, 253], [126, 232, 143, 248], [136, 225, 152, 241], [154, 216, 164, 226], [165, 209, 179, 219], [166, 220, 174, 230], [152, 229, 165, 237]]
Image name:  airplane_407.csv
Number of labels:  3
Labels:  [[30, 90, 78, 131], [143, 6, 190, 46], [200, 180, 253, 236]]
Image name:  airplane_343.csv
Number of labels:  2
Labels:  [[122, 153, 197, 247], [79, 5, 147, 98]]
Image name:  airplane_520.csv
Number of labels:  4
Labels:  [[63, 116, 92, 137], [107, 91, 132, 115], [140, 128, 165, 156], [176, 230, 202, 254]]
Image name:  airplane_296.csv
Number of labels:  2
Labels

In [23]:
#we pass each image for selective search and calculate iou for each bb output of ss out of 2000 proposals
for img in os.listdir(images_path):
    try:
        if img.split("_")[0] == "airplane":
            image = cv2.imread(os.path.join(images_path, img))
            ss.setBaseImage(image)
            ss.switchToSelectiveSearchFast(base_k=10)
            ssresults = ss.process()
            imout = image.copy()
            print("len of ssresults: ", len(ssresults))
    except Exception as e:
        print(e)
        print("Error in image: ", img)
        continue

len of ssresults:  1024
len of ssresults:  1180
len of ssresults:  1137
len of ssresults:  1235
len of ssresults:  1305
len of ssresults:  1282
len of ssresults:  1160
len of ssresults:  1102
len of ssresults:  1145
len of ssresults:  1422
len of ssresults:  1343
len of ssresults:  1111
len of ssresults:  1110
len of ssresults:  1151
len of ssresults:  1519
len of ssresults:  1074
len of ssresults:  1184
len of ssresults:  1161
len of ssresults:  1243
len of ssresults:  1169
len of ssresults:  1068
len of ssresults:  1088
len of ssresults:  1161
len of ssresults:  1322
len of ssresults:  1586
len of ssresults:  1035
len of ssresults:  1295
len of ssresults:  1068
len of ssresults:  1244
len of ssresults:  1174


KeyboardInterrupt: 